In [ ]:
import pypsa
import numpy as np
import pickle
from pathlib import Path

from _helpers import configure_logging
import logging
logger = logging.getLogger(__name__)

from pypsa.linopt import define_constraints, linexpr, get_var

def extra_functionality(n, snapshots):
    """... for solving the network; from pypsa-eur-sec."""
    add_battery_constraints(n)
    
    if n.name == 'LOHC shipping':
        add_LOHC_shipping_constraints(n)
        add_LOHC_chemical_constraint(n)
        
def add_battery_constraints(n):
    """Constraint for battery inverter capacity.
    
    Assumed battery inverter is bidirectional.
    Enforce same capacity for charging/discharging link.
    
    From pypsa-eur-sec. (2020-12-14)
    """
    
    chargers = n.links.query("name.str.startswith('battery inverter') "
                    "and bus1.str.startswith('battery') "
                    "and p_nom_extendable", engine='python').index
    dischargers = n.links.query("name.str.startswith('battery inverter') "
                        "and bus0.str.startswith('battery') "
                        "and p_nom_extendable", engine='python').index

    link_p_nom = get_var(n, "Link", "p_nom")
    lhs = linexpr((1,link_p_nom[chargers]), (-1, link_p_nom[dischargers].values))     
    pypsa.linopt.define_constraints(n, lhs, "=", 0, 'Link', 'charger_ratio')

def add_LOHC_shipping_constraints(n):
    """Constraint for shipping of LOHC to ensure consistent cargo capacity.
    
    LOHC requires an additional cargo store per convoy to store used LOHC produced
    during the journey or transported from the importer back to the exporter during
    the inbound journey.
    This constraint ensures that this LOHC (used) store is of the same size as the cargo
    store, i.e. the ships size is determined by the cargo store and the LOHC (used) store
    may not exceed this capacity.
    """

    lohc_stores = n.stores.filter(like='cargo LOHC (used)', axis=0).index
    cargo_stores = lohc_stores.str.replace('cargo LOHC \(used\)', 'cargo (exp)')

    stores_e_nom = get_var(n, "Store", 'e_nom')

    lhs = linexpr((1, stores_e_nom[cargo_stores]),
                  (-1., stores_e_nom[lohc_stores].values))

    pypsa.linopt.define_constraints(n, lhs, '=', 0, 'Store', 'lohc_shipping_constraint')
    
def add_LOHC_chemical_constraint(n):
    
    # for DBT: 0.944t LOHC per 1 t of loaded LOHC
    lohc_dbt_share = n.links.loc['LOHC dehydrogenation (imp)','efficiency2']
    
    loaded_stores = n.stores.filter(like='LOHC unloaded',axis=0).index.union(
        n.stores.filter(regex='LOHC transport ship convoy \d+ cargo \(exp\)',axis=0).index
    )
    
    unloaded_stores = n.stores.filter(like='LOHC loaded',axis=0).index.union(
        n.stores.filter(regex='LOHC transport ship convoy \d+ cargo LOHC \(used\)',axis=0).index
    )
        
    generator_p_nom = get_var(n, "Generator", 'p_nom')
    stores_e = get_var(n, "Store", 'e')

    for s in n.snapshots:

        lhs = linexpr((1, stores_e.loc[s, unloaded_stores].values),
                      (lohc_dbt_share, stores_e.loc[s, loaded_stores].values)).sum()
        lhs += linexpr((-1,generator_p_nom['LOHC chemical (exp)']))[0]
        
        pypsa.linopt.define_constraints(n, lhs, '<=', 0, 'Generator', f'lohc_chemical__constraint_{s}')

In [ ]:
if __name__ == "__main__":

    configure_logging(snakemake)
    
    for op in ['default', 'fallback']:
        solver_options = snakemake.config['solver'][op].copy()
        solver_name = solver_options.pop('name')

        # Load additional components
        with open(snakemake.input["additional_components"], "rb") as f:
            override_component_attrs = pickle.load(f)
        network = pypsa.Network(override_component_attrs=override_component_attrs)

        network.import_from_netcdf(snakemake.input["network"])

        network.consistency_check()

        logger.info(f'Solving network using solver options: {solver_options}.')
        logger.info("Starting LOPF.")
        status, termination_condition = network.lopf(snapshots=network.snapshots,
                                                     extra_functionality=extra_functionality,
                                                     pyomo=False,
                                                     solver_name=solver_name,
                                                     solver_options=solver_options,
                                                     solver_logfile=snakemake.log['python'])
        logger.info("End of LOPF.")
        
        if status == 'ok':
            network.export_to_netcdf(snakemake.output["network"])
            break
        elif status == 'warning' and op == 'default':
            logger.info(f'Optimsation ended with status {status}. Retrying with fallback solver config.')
        else:
            logger.warning(f'Optimisation ended with unexpected status: {status}. '
                           f'Unoptimised network not saved.')